In [1]:
import os
from typing import List
from pydantic import BaseModel
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END

In [2]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
llm=init_chat_model("openai:gpt-4o")

In [3]:
### Load And Embed Documents\n",
docs = TextLoader("sample_docs.txt", encoding="utf-8").load()
chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)
vectorstore = FAISS.from_documents(chunks, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [4]:
### Define Agent State
class IterativeRAGState(BaseModel):
    question: str
    refined_question: str = ""
    retrieved_docs: List[Document] = []
    answer: str = ""
    verified: bool = False
    attempts: int = 0

In [5]:
### Retrieve Node
def retrieve_docs(state: IterativeRAGState) -> IterativeRAGState:
        query = state.refined_question or state.question
        docs = retriever.invoke(query)
        return state.model_copy(update={"retrieved_docs": docs})

In [8]:
### Reflect And Verify\n",
def generate_answer(state: IterativeRAGState) -> IterativeRAGState:
    context = "".join(doc.page_content for doc in state.retrieved_docs)
    prompt = f"""Use the following context to answer the question:
    Context:
    {context}
    Question:
    {state.question}
    """
    response = llm.invoke(prompt.strip()).content.strip()
    return state.model_copy(update={"answer": response, "attempts": state.attempts + 1})

In [9]:
## Reflect on answer\n",
def reflect_on_answer(state: IterativeRAGState) -> IterativeRAGState:
    prompt = f"""
    Evaluate whether the answer below is factually sufficient and complete.
    Question: {state.question}
    Answer: {state.answer}
    Respond 'YES' if it's complete, otherwise 'NO' with feedback.
    """
    feedback = llm.invoke(prompt).content.lower()
    verified = "yes" in feedback
    return state.model_copy(update={"verified": verified})

In [11]:
## Refine query\n",
def refine_query(state: IterativeRAGState) -> IterativeRAGState:
    prompt = f"""
    The answer appears incomplete. Suggest a better version of the query that would help retrieve more relevant context.
    Original Question: {state.question}
    Current Answer: {state.answer}
    """
    new_query = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"refined_question": new_query})

In [12]:
builder = StateGraph(IterativeRAGState)
builder.add_node("retrieve", retrieve_docs)
builder.add_node("answer", generate_answer)
builder.add_node("reflect", reflect_on_answer)
builder.add_node("refine", refine_query)
builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "answer")
builder.add_edge("answer", "reflect")
builder.add_conditional_edges(
    "reflect",
       lambda s: END if s.verified or s.attempts >= 2 else "refine"
)
builder.add_edge("refine", "retrieve")
builder.add_edge("answer", END)
graph = builder.compile()

In [ ]:
query = "agent loops  and transformer-based systems?"
initial_state = IterativeRAGState(question=query)
final = graph.invoke(initial_state)
print("✅ Final Answer:", final["answer"])
print("🧠 Verified:", final["verified"])
print("🔁 Attempts:", final["attempts"])